# 使用sklearn的多层感知机

实验内容：  
1. 学习标准化处理的方法
2. 使用sklearn.neural_network.MLPClassifier完成Breast_Cancer_Wisconsin结果预测问题
3. 绘制学习率为3，1，0.1，0.01训练集损失函数的变化曲线

## 1. 读取数据集

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('data/Breast_Cancer_Wisconsin/data')
print(data.shape)

In [ ]:
data = data.values 
data_x = data[:,2:-1]
data_y = data[:,1:2]
data_y = np.reshape(data_y,(-1))

print(data_x.shape)
print(data_y.shape)

## 2. 划分数据集

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
trainX, testX, trainY, testY = train_test_split(data_x, data_y, test_size = 0.4, random_state = 32)

In [ ]:
trainX.shape, trainY.shape, testX.shape, testY.shape

## 3. 数据预处理

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
standard = StandardScaler()
trainX = standard.fit_transform(trainX)
testX = standard.transform(testX)

## 4. 引入模型

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
model = MLPClassifier(solver = 'sgd', learning_rate = 'constant', momentum = 0, learning_rate_init = 0.1, max_iter = 500)
model.fit(trainX, trainY)
prediction = model.predict(testX)

## 5. 预测与评估

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(prediction, testY)

精度达到了97.8%

## 6. 绘制训练集损失函数值的变化曲线

我们可以通过model.loss_curve_获取模型在训练过程中，损失函数损失值的变化曲线

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (10, 6))
plt.plot(model.loss_curve_)
plt.xlabel('epoch')
plt.ylabel('loss')

可以看到随着迭代轮数的增加，loss降低地越来越缓慢

## test：请你在一张图内，绘制出学习率为3，学习率为1，学习率为0.1，学习率为0.01，四个模型的损失函数变化曲线，最大迭代轮数为250轮。

提示：分别训练4个模型，然后在一张图中分别绘制4个模型的loss_curve_即可。

In [ ]:
plt.figure(figsize = (10, 10))
# YOUR CODE HERE
model1 = MLPClassifier(solver='sgd', learning_rate='constant', momentum = 0, learning_rate_init=3, max_iter=500)
model1.fit(trainX, trainY)
model2 = MLPClassifier(solver='sgd', learning_rate='constant', momentum = 0, learning_rate_init=1, max_iter=500)
model2.fit(trainX, trainY)
model3 = MLPClassifier(solver='sgd', learning_rate='constant', momentum = 0, learning_rate_init=0.1, max_iter=500)
model3.fit(trainX, trainY)
model4 = MLPClassifier(solver='sgd', learning_rate='constant', momentum = 0, learning_rate_init=0.01, max_iter=500)
model4.fit(trainX, trainY)
plt.figure(figsize=(10, 6))
plt.plot(model1.loss_curve_, label='learning rate=3')
plt.plot(model2.loss_curve_, label='learning rate=1')
plt.plot(model3.loss_curve_, label='learning rate=0.1')
plt.plot(model4.loss_curve_, label='learning rate=0.01')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()